#### Music algorithm

In [37]:
# mapping from 3 coordinates to 1 (opposite of unravel)
def ravel_index(x, dims):
    i = 0
    for dim, j in zip(dims, x):
        i *= dim
        i += j
    return i
positionsY=data['px_grid']
print(positionsY)

[[-0.63162207 -0.48892353  0.60167046]
 [-0.63127202 -0.48937541  0.60167046]
 [-0.63092165 -0.48982704  0.60167046]
 ...
 [-0.66759104 -0.52290558  0.52999241]
 [-0.66721667 -0.52338319  0.52999241]
 [-0.66684195 -0.52386053  0.52999241]]


In [62]:
def music(S, positionsY=data['px_grid']):
    '''
    params:
    S: visibility matrix. shape = (n_timeframes, n_antenna, n_antenna): correlation matrix 
    '''
    N = 3
    M = 21
    P = []
    c0 = 3e8;#speed of light 
    fStop = 77e9;
    fStrt = 76e9;
    r=[];

    for x in range(S.shape[0]): 
        # finding eigen values and eigen vectors
        eigvals, eigvecs = np.linalg.eig(S[x,:,:])
        #print("Covariance Matrix S : ", S[0,:,:].shape)
        # sorting eig vals and eig vecs in decreasing order of eig vals
        idx = eigvals.argsort()[::-1]
        eigvals = eigvals[idx]
        eigvecs = eigvecs[:, idx]
        #print("Eigenvalues: ", eigvals)
        #print("Eigenvectors: ", eigvecs)
        #plt.plot(eigvals)
        #For each x, there always 3 leading eigenvalues which will be taken for the sources and the rest for the noise
        # separating source and noise eigvectors
        E, En = eigvecs[:, :N], eigvecs[:, N:]
        print(En.shape)
        #Us: signal matrix: (24,3)
        #Un: noise matrix: (24,21)
        #calculate P
        # MUSIC search directions
        AzSearch = np.arange(-90, 90, 0.1); # Azimuth values to search
        print(AzSearch.shape)
        ElSearch = [0];#placeholder, we do not do elevation
        NrChn = 24 #number of antennas
        for i in range (NrChn):
            r.append( [0.,ravel_index(positionsY[i],positionsY[i].shape)/1000.,0.]);
        # Wavenumber vectors (in units of wavelength/2)
        X1 = np.cos(np.multiply(AzSearch, np.pi / 180.)); 
        X2 = np.sin(np.multiply(AzSearch, np.pi / 180.));
        X3 = np.sin(np.multiply(AzSearch, 0.));
        kSearch  = np.multiply([X1, X2, X3], 2*np.pi/ (c0 / ((fStop+fStrt)/2)));
        print(kSearch.shape)
        ku = np.dot (r,kSearch);
        print(ku.shape)
        ASearch = np.exp(np.multiply(ku, -1j));
        print(ASearch.shape)
        chemodan = np.dot(np.transpose(ASearch), En);    
        aac = np.absolute(chemodan);
        aad = np.square(aac);
        aae = np.sum(aad,1);
        Z = aae;
        # Get spherical coordinates
        P = np.unravel_index(Z.argmin(), Z.shape);
        print(AzSearch[P]);
        return AzSearch, Z

In [63]:
music(data['S'])

(24, 21)
(1800,)
(3, 1800)
(24, 1800)
(24, 1800)
89.89999999998977


(array([-90. , -89.9, -89.8, ...,  89.7,  89.8,  89.9]),
 array([21.80609426, 21.80609426, 21.80609426, ..., 21.80523179,
        21.80523179, 21.80523178]))